# RAG - ensemble (Korean EMB and English TEXT)
* Container: Data Science 3.0 (studio, python 3.10)

## 0. Install packages and Setup env

In [8]:
import sys

In [9]:
%load_ext autoreload
%autoreload 2
sys.path.append('../utils') # src 폴더 경로 설정

In [3]:
install_needed = True  # should only be True once

In [4]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U sagemaker
    !{sys.executable} -m pip install -U langchain
    !{sys.executable} -m pip install -U faiss-cpu
    
    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


### 1. SageMaker Endpoint Wrapper

### 1.1 SageMaker LLM_TEXT Wrapper

In [10]:
import json
import boto3
import numpy as np
from typing import Any, Dict, List, Optional
from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.llms.sagemaker_endpoint import LLMContentHandler, SagemakerEndpoint
from langchain.embeddings.sagemaker_endpoint import EmbeddingsContentHandler

In [11]:
class AI21ContexualAnswerContentHandler(LLMContentHandler):
    
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
        
        context, question = prompt.split("||SPEPERATOR||")
        #print ("context:",  context)
        #print ("question:",  question)
        input_str = json.dumps({"context": context, "question":question})
        return input_str.encode('utf-8')

    def transform_output(self, output: bytes) -> str:
        response_json = output.read()
        res = json.loads(response_json)
        return res['answer']

In [12]:
aws_region = boto3.Session().region_name
LLMTextContentHandler = AI21ContexualAnswerContentHandler()
endpoint_name_text = "contextual-answers-2023-06-19-01-57-17"
seperator = "||SPEPERATOR||"

In [13]:
llm_text = SagemakerEndpoint(
    endpoint_name=endpoint_name_text,
    region_name=aws_region,
    content_handler=LLMTextContentHandler,
)

### 1.2 SageMaker LLM_EMB Wrapper

In [14]:
class SagemakerEndpointEmbeddingsJumpStart(SagemakerEndpointEmbeddings):
    def embed_documents(self, texts: List[str], chunk_size: int=1) -> List[List[float]]:
        """Compute doc embeddings using a SageMaker Inference Endpoint.

        Args:
            texts: The list of texts to embed.
            chunk_size: The chunk size defines how many input texts will
                be grouped together as request. If None, will use the
                chunk size specified by the class.

        Returns:
            List of embeddings, one for each text.
        """
        results = []
        _chunk_size = len(texts) if chunk_size > len(texts) else chunk_size
        
        print("text size: ", len(texts))
        print("_chunk_size: ", _chunk_size)

        for i in range(0, len(texts), _chunk_size):
            
            #print (i, texts[i : i + _chunk_size])
            response = self._embedding_func(texts[i : i + _chunk_size])
            #print (i, response, len(response[0].shape))
            
            results.extend(response)
        return results

In [15]:
class KoSimCSERobertaContentHandler(EmbeddingsContentHandler):
    
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        
        input_str = json.dumps({"inputs": prompt, **model_kwargs})
        
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        
        response_json = json.loads(output.read().decode("utf-8"))
        ndim = np.array(response_json).ndim    
        
        if ndim == 4:
            # Original shape (1, 1, n, 768)
            emb = response_json[0][0][0]
            emb = np.expand_dims(emb, axis=0).tolist()
        elif ndim == 2:
            # Original shape (n, 1)
            emb = []
            for ele in response_json:
                e = ele[0][0]
                emb.append(e)
        else:
            print(f"Other # of dimension: {ndim}")
            emb = None
        return emb

In [16]:
LLMEmbHandler = KoSimCSERobertaContentHandler()
endpoint_name_emb = "KoSimCSE-roberta-2023-06-27-14-15-09"

In [17]:
llm_emb = SagemakerEndpointEmbeddingsJumpStart(
    endpoint_name=endpoint_name_emb,
    region_name=aws_region,
    content_handler=LLMEmbHandler,
)

**Now, we can build an QA application. <span style="color:red">LangChain makes it extremly simple with following few lines of code</span>.**

## 4. Vector Store
FAISS Vector Store
- https://python.langchain.com/docs/modules/data_connection/vectorstores/integrations/faiss <BR>

OpenSearch
- https://python.langchain.com/docs/modules/data_connection/vectorstores/integrations/opensearch

In [18]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import Chroma, AtlasDB, FAISS
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

### 4.1 load context files and build indexer

#### Increase csv field size

In [19]:
import csv
csv.field_size_limit(100000000)
csv.field_size_limit()

100000000

In [20]:
loader = CSVLoader(
    file_path="../dataset/quenstion_answer_ko.csv",
    source_column="Source",
    encoding="utf-8"
)
context_documents = loader.load()

In [21]:
len(context_documents), context_documents[5]

(8493,
 Document(page_content='Information: 네, 이마트 창동점에는 포토센터(사진관)가 있고, 전화번호는 053-755-1559, 운영시간은 10:00~20:00 입니다.<sep>이마트 창동점\nSource: 이마트 창동점', metadata={'source': '이마트 창동점', 'row': 5}))

#### Create Indexer
TextSplitter: https://js.langchain.com/docs/modules/indexes/text_splitters/examples/recursive_character

In [22]:
index_creator = VectorstoreIndexCreator(
    vectorstore_cls=FAISS,
    embedding=llm_emb,
    #text_splitter=CharacterTextSplitter(chunk_size=700, chunk_overlap=0),
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
)

## show documents
#text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
#docs = text_splitter.split_documents(context_documents)

In [23]:
%%time
index = index_creator.from_loaders([loader])

text size:  8535
_chunk_size:  1
CPU times: user 3min 5s, sys: 4.92 s, total: 3min 10s
Wall time: 11min 19s


#### Save and Read index from local

In [24]:
# Save
index.vectorstore.save_local("../indexer/indexer_ko_modified")

In [25]:
# Read
index_ = FAISS.load_local("../indexer/indexer_ko_modified", llm_emb)

#### Check documents

In [30]:
#index.vectorstore.index_to_docstore_id
#index.vectorstore.docstore._dict
#context_documents[4].page_content
index_.index_to_docstore_id
index_.docstore._dict

{'977fb818-c15c-4668-a15b-389542d90563': Document(page_content='Information: 이마트 창동점의 우편번호는 다음과 같습니다. \n132045<sep>이마트 창동점\nSource: 이마트 창동점', metadata={'source': '이마트 창동점', 'row': 0}),
 'f35d49ae-cc8a-4938-83be-0f7e41fd6dcb': Document(page_content='Information: 이마트 창동점의 주소는 다음과 같습니다. \n서울 도봉구 노해로 65길 4<sep>이마트 창동점\nSource: 이마트 창동점', metadata={'source': '이마트 창동점', 'row': 1}),
 'a7f03b99-d10d-4ba3-a9bf-81d2f2d5c60d': Document(page_content='Information: 이마트 창동점의 전화번호는 다음과 같습니다. \n02-901-1234<sep>이마트 창동점\nSource: 이마트 창동점', metadata={'source': '이마트 창동점', 'row': 2}),
 '5d6d49ca-5711-47a2-85cc-d0c90bd9185d': Document(page_content='Information: 이마트 창동점의 휴일은 다음과 같습니다/. \n2023-07-09, 2023-07-23, 2023-06-11, 2023-06-25, 2023-01-22<sep>이마트 창동점\nSource: 이마트 창동점', metadata={'source': '이마트 창동점', 'row': 3}),
 '316a4082-25b3-4862-bc4b-e3f465e375c8': Document(page_content='Information: 이마트 창동점의 주차정보, 주차요금, 무료주차 정보는 다음과 같습니다. &lt;유료주차 운영 안내&gt;- 24시간운영- 무료회차 시간 30분이내- 기본요금: 10분당 1,000원- 30분 초과시 회차시간 포함 요

In [7]:
context_documents
for idx, key in enumerate(context_documents):
    if key.metadata["source"] == "노브랜드 원주단구점":
        print(key)
        print("==")

page_content='Information: 노브랜드 원주단구점의 우편번호는 다음과 같습니다. \n26490<sep>노브랜드 원주단구점\nSource: 노브랜드 원주단구점' metadata={'source': '노브랜드 원주단구점', 'row': 1235}
==
page_content='Information: 노브랜드 원주단구점의 주소는 다음과 같습니다. \n강원 원주시 단구로 355 (단구동)<sep>노브랜드 원주단구점\nSource: 노브랜드 원주단구점' metadata={'source': '노브랜드 원주단구점', 'row': 1236}
==
page_content='Information: 노브랜드 원주단구점의 전화번호는 다음과 같습니다. \nnan<sep>노브랜드 원주단구점\nSource: 노브랜드 원주단구점' metadata={'source': '노브랜드 원주단구점', 'row': 1237}
==
page_content='Information: 노브랜드 원주단구점의 휴일은 다음과 같습니다/. \n2023-07-12, 2023-07-26, 2023-06-14, 2023-06-28, 2023-01-22<sep>노브랜드 원주단구점\nSource: 노브랜드 원주단구점' metadata={'source': '노브랜드 원주단구점', 'row': 1238}
==
page_content='Information: 노브랜드 원주단구점의 주차정보, 주차요금, 무료주차 정보는 다음과 같습니다. \nnan\n 전체 매장 주차가능 수: 13대<sep>노브랜드 원주단구점\nSource: 노브랜드 원주단구점' metadata={'source': '노브랜드 원주단구점', 'row': 1239}
==


In [32]:
for idx, key in enumerate(index_.docstore._dict):
    record = index_.docstore._dict[key]

    page_content = record.page_content
    source = record.metadata["source"]

    if source == "이마트 동탄점":
        print(record)
        print("==")

page_content='Information: 이마트 동탄점의 우편번호는 다음과 같습니다. \n445170<sep>이마트 동탄점\nSource: 이마트 동탄점' metadata={'source': '이마트 동탄점', 'row': 233}
==
page_content='Information: 이마트 동탄점의 주소는 다음과 같습니다. \n경기도 화성시 동탄중앙로 376<sep>이마트 동탄점\nSource: 이마트 동탄점' metadata={'source': '이마트 동탄점', 'row': 234}
==
page_content='Information: 이마트 동탄점의 전화번호는 다음과 같습니다. \n031-647-1234<sep>이마트 동탄점\nSource: 이마트 동탄점' metadata={'source': '이마트 동탄점', 'row': 235}
==
page_content='Information: 이마트 동탄점의 휴일은 다음과 같습니다/. \n2023-07-09, 2023-07-23, 2023-06-11, 2023-06-25, 2023-01-22<sep>이마트 동탄점\nSource: 이마트 동탄점' metadata={'source': '이마트 동탄점', 'row': 236}
==
page_content='Information: 이마트 동탄점의 주차정보, 주차요금, 무료주차 정보는 다음과 같습니다. 유료주차 이용안내(사전/무인정산기)시행일자 : 2019년 7월 1일(월요일)[일반요금]1. 30분 회차 - 무료 ※30분 초과 시 회차시간 포함 요금징수2. 기본 10분 - 1,000원3. 운영시간 - 10:00 ~ 23:00[고객할인]1. 1만원이상 - 2시간2. 3만원 이상 - 3시간3. 5만원 이상 - 4시간※최대 4시간을 초과 할 수 없습니다.(영수증 중복적용 가능)<sep>이마트 동탄점\nSource: 이마트 동탄점' metadata={'source': '이마트 동탄점', 'row': 237}
==
page_content='Information: 이마트 동

## 5. QnA

In [33]:
from functools import lru_cache
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain

In [34]:
translate = boto3.client("translate")

Python cache
* https://www.daleseo.com/python-cache/

In [35]:
@lru_cache(maxsize=10000)
def trans(text, target="en"):

    response=translate.translate_text(
        Text=text,
        SourceLanguageCode="Auto",
        TargetLanguageCode=target
    )

    text_translate = response["TranslatedText"]
    #print (text_translate)
    return text_translate

### 5.1 Query and Response

In [36]:
import copy
import functools
import concurrent.futures

In [37]:
prompt_template = ''.join(["{context}", seperator, "{question}"])
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain = load_qa_chain(llm=llm_text, chain_type="stuff", prompt=PROMPT, verbose=True)

In [42]:
def get_similiar_docs(query, k=10, fetch_k=100, score=False, store=""):
    
    query = f'{store}, {query}'
    print (query)
    
    if score: 
        #similar_docs = index.vectorstore.similarity_search_with_score(query, k=k, fetch_k=fetch_k, filter=dict(source=store))
        similar_docs = index_.similarity_search_with_score(
            query,
            k=k,
            fetch_k=fetch_k,
            filter=dict(source=store)
        )
    else:
        #similar_docs = index.vectorstore.similarity_search(query, k=k, fetch_k=fetch_k, filter=dict(source=store))
        similar_docs = index_.similarity_search(
            query,
            k=k,
            fetch_k=fetch_k,
            filter=dict(source=store)
        )
        
    similar_docs_copy = copy.deepcopy(similar_docs)
    return similar_docs_copy

def worker(target, args):
    
    doc = args
    #print ("worker", doc, target)
    doc.page_content = trans(doc.page_content, target=target)
    
    return doc

function = functools.partial(worker, "en") # 반복되는 것은 먼저 쓰기

def get_answer(query, store="", k=5):
    
    similar_docs = get_similiar_docs(query, k=k, store=store)    
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        similar_docs = list(executor.map(function, [(doc) for doc in similar_docs]))
    #for doc in similar_docs: doc.page_content = trans(doc.page_content)
    
    answer = trans(chain.run(input_documents=similar_docs, question=trans(query)), target="ko")
    
    return answer



In [43]:
%%time
question ="이마트 동탄점 무료주차 하려면 얼마나 사야해?"
response = get_answer(question, store="이마트 동탄점", k=5)

이마트 동탄점, 이마트 동탄점 무료주차 하려면 얼마나 사야해?


> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Information: The parking information, parking fees, and free parking information at E-Mart Dongtan is as follows. Paid parking user guide (advance/unmanned vending machine) Effective date: July 1, 2019 (Monday) [Regular fee] 1. 30 minute round trip - free ※If it exceeds 30 minutes, the fee collection includes the tour time 2. Basic 10 minutes - 1,000 won 3. Opening hours - 10:00 to 23:00 [Customer discount] 1. 10,000 won or more - 2 hours 2. 30,000 won or more - 3 hours 35,000 won or more - 4 hours ※The maximum period cannot exceed 4 hours. <sep>(Receipts can be applied in duplicate) E-Mart Dongtan Branch
Source: E-Mart Dongtan

Information: The parking information, parking fees, and free parking information at E-Mart Dongtan is as follows. 
[Note] 1. Only same-day receipts issued at the E-Mart checkout counter can be used.2. For other receipts, please apply the parking dis

In [44]:
print (f'question: {question}')
print (f'response: {response}')
print (f'cache_info: {trans.cache_info()}')

question: 이마트 동탄점 무료주차 하려면 얼마나 사야해?
response: 1만원 이상 - 2시간
3만원 이상 - 3시간
3만 5천원 이상 - 4시간
cache_info: CacheInfo(hits=8, misses=8, maxsize=10000, currsize=8)
